### This notebook shows how to 

##### 1. Build a RAG index from raw documents folder and store in Faiss vector database 
##### 2. Query from RAG index
##### 3. Demo of answers shared by LLM + RAG

In [2]:
from llama_index.core import SimpleDirectoryReader
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
import faiss

# Load your documents
documents = SimpleDirectoryReader(input_dir="../data/docs/").load_data()

### 1. Load documents for indexing

In [3]:
# Initialize embeddings, used for encoding documents into embedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

/home/nausheenfatma/.cache/pypoetry/virtualenvs/canadaimmigrationhelpdesk-usinggenai-s0qUKeX3-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


### 2. Emded the documents and store in FAISS vectorstore index

In [4]:
# Create FAISS index
dimension = 384  # Dimension for MiniLM embeddings
faiss_index = faiss.IndexFlatL2(dimension)

# Construct the FaissVectorStore
vector_store = FaissVectorStore(faiss_index=faiss_index)


# Create a StorageContext to use with LlamaIndex
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Build index
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, embed_model=embed_model)
retriever = index.as_retriever()

### 3. Initialize Llama model for text generation from retrieved documents

In [15]:
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama2", request_timeout=100.0)

# Query function
def query_rag_system(question):
    retrieved_docs = retriever.retrieve(question)
    context = "\n".join([doc.text for doc in retrieved_docs])    
    prompt = f"Context:\n{context}\n\n Question: {question}\n\n\n\nAnswer:"
    response = llm.complete(prompt)
    return response


### 4. Query from domain specific RAG index

In [16]:
# Example query
question = "Who can apply for open work permit?"
answer = query_rag_system(question)
print(answer)


To be eligible to apply for an open work permit through the Agri-Food Pilot, you must meet certain requirements. These include:

* Being a foreign worker who is currently working in Canada on a work permit, and
* Having a job offer from an employer in one of the eligible industries or occupations listed by Immigration, Refugees and Citizenship Canada (IRCC), and
* Meeting the requirements for the Agri-Food Pilot program.

If you meet these requirements, you can apply for an open work permit through the Agri-Food Pilot. Your family members may also be eligible to apply for an open work permit, regardless of their job skill level. However, if they are in a TEER 2 or 3 occupation, they must meet additional requirements to qualify.

It's important to note that the application process and requirements may vary depending on your individual circumstances. It's recommended to consult with IRCC or a qualified immigration lawyer to ensure you have the most up-to-date information and to get profe

In [17]:
question = "Can I apply for the Rural and Northern Immigration Pilot now?"

In [ ]:
answer = query_rag_system(question)
print(answer)

Unfortunately, the Rural and Northern Immigration Pilot has ended on August 31, 2024. As a result, new applications are no longer being accepted. However, if you were eligible and applied before the deadline, your application is still being processed. Additionally, if you have an eligible job offer, you may be able to apply for a work permit while your permanent residence application is being processed.

It's important to note that the Agri-Food Pilot is still open and accepting applications. If you are interested in immigrating to Canada through this program, you can find more information on the Immigration, Refugees and Citizenship Canada website.

In terms of work permits for family members, they may be eligible for an open work permit if certain requirements are met. Your family member can apply for an open work permit online through the Immigration, Refugees and Citizenship Canada website.

I hope this information is helpful. If you have any further questions, please don't hesitat

In [21]:
question = "Can I drive in Canada after moving?"

In [22]:
answer = query_rag_system(question)
print(answer)

Welcome to the Canada.ca page on driving in Canada! Here's an overview of the information provided on this page:

Driving Licences:
If you're a newcomer to Canada, you may be able to use your foreign driver's licence for a short time after arriving. However, you will need to get a Canadian driver's licence issued by the government of the province or territory where you live. The process for obtaining a driver's licence in Canada varies depending on your driving background and the province or territory where you live.

Car Insurance:
It is illegal to drive without car insurance in Canada. You must get insurance coverage for yourself and any passengers in your vehicle, as well as for any damage to your vehicle or other vehicles or property in an accident. There are different types of car insurance plans available, including coverage for injuries to yourself or damage to your vehicle, or both. The cost of car insurance varies depending on the province or territory where you live and your 

In [23]:
question = "What kind of Canadian laws and rules I should be aware of?"

In [24]:
answer = query_rag_system(question)
print(answer)

As a new immigrant to Canada, it's important to be aware of the country's laws and regulations to ensure a smooth transition and avoid any legal issues. Here are some key areas of Canadian law that you may find useful to know:

1. Criminal Law: Understanding the laws surrounding criminal activity, such as assault, theft, and drug offenses, is essential for your safety and well-being in Canada.
2. Administrative Law: This area of law deals with government rules and regulations, including those related to immigration, citizenship, and public services.
3. Civil Law: Civil laws govern disputes between individuals, organizations, or the government. You may encounter civil laws in situations such as contract disputes, property rights, and family law.
4. Labour Laws: As a worker in Canada, you should be aware of labour laws related to work hours, wages, health and safety, and employment standards.
5. Public Health Laws: Familiarizing yourself with public health laws and regulations can help y